In [53]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [54]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

RuntimeError: Physical devices cannot be modified after being initialized

In [ ]:
for study in os.listdir("Studies/"):
    for orientation in os.listdir("Studies/"+study+"/"):
        for folder in os.listdir("Studies/"+study+"/"+orientation+"/"):
            if folder == "anno":
                df = pd.read_csv("Studies/"+study+"/"+orientation+"/"+folder+"/gTruth.csv").dropna().reset_index().drop(columns="index")
                df["PixelLabelData"] = df["PixelLabelData"].map(lambda path: "/".join(["Studies"]+path.split("/")[-5:]))
                df["OriginalData"] = ""
                try:
                    os.mkdir("Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/")
                    vidcap = cv2.VideoCapture("Studies/"+study+"/"+orientation+"/clip/"+orientation+".mp4")
                    for i in range(len(df)):
                        name = df.at[i, 'PixelLabelData'].split("/")[-1]
                        time = round(float(df.at[i, 'Time'].split(" ")[0])*1000)
                        vidcap.set(cv2.CAP_PROP_POS_MSEC, time)
                        success,image = vidcap.read()
                        if success:
                            cv2.imwrite("Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/"+name, image)
                            df.at[i, "OriginalData"] = "Studies/"+study+"/"+orientation+"/"+folder+"/OriginalData/"+name
                    df = df.replace('', np.nan).dropna().reset_index().drop(columns="index")
                    df.to_csv("Studies/"+study+"/"+orientation+"/"+"data.csv", index=False)
                except:
                    print("folder done")

In [ ]:
finaldf = []
for study in os.listdir("Studies/"):
    for orientation in os.listdir("Studies/"+study+"/"):
        for fsobject in os.listdir("Studies/"+study+"/"+orientation+"/"):
            if fsobject == "data.csv":
                df = pd.read_csv("Studies/"+study+"/"+orientation+"/"+fsobject)
                finaldf.append(df)
finaldf = pd.concat(finaldf, ignore_index=True)
finaldf.to_csv("dataset.csv", index=False)

In [ ]:
dataset = pd.read_csv("dataset.csv")
try:
    os.mkdir("data/")    
    os.mkdir("data/train_frames/")
    os.mkdir("data/mask_frames/")
    os.mkdir("data/train_frames/train")
    os.mkdir("data/mask_frames/mask")
    for i in range(len(dataset)):
        image = cv2.imread(dataset.at[i, 'OriginalData'])
        mask = cv2.imread(dataset.at[i, 'PixelLabelData'], cv2.IMREAD_GRAYSCALE)
        if (sum(sum(mask)) > 0):
            cv2.imwrite("data/train_frames/train/"+str(i)+".png", image)
            cv2.imwrite("data/mask_frames/mask/"+str(i)+".png", mask)
except:
    for i in range(len(dataset)):
        image = cv2.imread(dataset.at[i, 'OriginalData'])
        mask = cv2.imread(dataset.at[i, 'PixelLabelData'], cv2.IMREAD_GRAYSCALE)
        if (sum(sum(mask)) > 0):
            cv2.imwrite("data/train_frames/train/"+str(i)+".png", image)
            cv2.imwrite("data/mask_frames/mask/"+str(i)+".png", mask)

In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

In [47]:
train_image_generator = train_datagen.flow_from_directory('data/train_frames/', target_size=(1024,1024), class_mode=None, batch_size=32)
train_mask_generator = train_datagen.flow_from_directory('data/mask_frames/', target_size=(1024,1024), class_mode=None, batch_size=32)

Found 100 images belonging to 1 classes.
Found 100 images belonging to 1 classes.


In [48]:
train_generator = zip(train_image_generator, train_mask_generator)

In [49]:
#U-net

inputs = tf.keras.layers.Input((1024,1024,3))

conv1 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
drop4 = tf.keras.layers.Dropout(0.5)(conv4)
pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = tf.keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = tf.keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
drop5 = tf.keras.layers.Dropout(0.5)(conv5)

up6 = tf.keras.layers.Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(drop5))
merge6 = tf.keras.layers.concatenate([drop4,up6], axis = 3)
conv6 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

up7 = tf.keras.layers.Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv6))
merge7 = tf.keras.layers.concatenate([conv3,up7], axis = 3)
conv7 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

up8 = tf.keras.layers.Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv7))
merge8 = tf.keras.layers.concatenate([conv2,up8], axis = 3)
conv8 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

up9 = tf.keras.layers.Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv8))
merge9 = tf.keras.layers.concatenate([conv1,up9], axis = 3)
conv9 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = tf.keras.layers.Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv10 = tf.keras.layers.Conv2D(1, 1, activation = 'sigmoid')(conv9)

model = tf.keras.Model(inputs = inputs, outputs = conv10)

model.compile(optimizer = tf.optimizers.Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [51]:
NO_OF_TRAINING_IMAGES = len(os.listdir('data/train_frames/train/'))

NO_OF_EPOCHS = 30

BATCH_SIZE = 32

weights_path = '/weights'

checkpoint = tf.keras.callbacks.ModelCheckpoint(weights_path, monitor='accuracy', 
                             verbose=1, save_best_only=True, mode='max')

csv_logger = tf.keras.callbacks.CSVLogger('./log.out', append=True, separator=';')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', verbose = 1,
                              min_delta = 0.01, patience = 3, mode = 'max')

callbacks_list = [checkpoint, csv_logger, earlystopping]

results = model.fit(train_generator, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          callbacks=callbacks_list)
model.save('Model.h5')

Epoch 1/30


ResourceExhaustedError:  OOM when allocating tensor with shape[32,64,1024,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_4/conv2d_107/Relu (defined at <ipython-input-50-b3fb14e2d88e>:19) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_6653]

Function call stack:
train_function
